<div class="alert block alert-info alert">

# <center> Scientific Programming in Python

## <center>Karl N. Kirschner<br>Bonn-Rhein-Sieg University of Applied Sciences<br>Sankt Augustin, Germany

# <center> Testing Code (and GenAI)

<!-- <br><br> -->


#### References
1. Fucci, D., Erdogmus, H., Turhan, B., Oivo, M. and Juristo, N., 2016. A dissection of the test-driven development process: Does it really matter to test-first or to test-last?. IEEE Transactions on Software Engineering, 43(7), pp.597-614. (https://ieeexplore.ieee.org/abstract/document/7592412)
2. Mock, M., Melegati, J. and Russo, B., 2024, June. Generative AI for test driven development: preliminary results. In International Conference on Agile Software Development (pp. 24-32). Cham: Springer Nature Switzerland. (https://link.springer.com/chapter/10.1007/978-3-031-72781-8_3)
3. Piya, S. and Sullivan, A., 2024, April. LLM4TDD: best practices for test driven development using large language models. In Proceedings of the 1st International Workshop on Large Language Models for Code (pp. 14-21). (https://dl.acm.org/doi/abs/10.1145/3643795.3648382)
4. Ardic, B., Dilavrec, Q.L. and Zaidman, A., 2025. How Students Use Generative AI for Software Testing: An Observational Study. arXiv preprint arXiv:2510.10551. (https://arxiv.org/abs/2510.10551)

<hr style="border:2px solid gray"></hr>

**Note**: All user-defined functions in the notebook do not include document strings (i.e. block comments) or internal checks. This is purposely done to focus on the teaching aspects of the lecture. **A full and proper user-defined function would include these.**

<center>
<div> <img src="00_images/ai/unit_testing_gemini_generated.png" width="400"/> </div>
Generated with Gemini AI ∙ October 27, 2025 at 16:20
</center>

# Context: The Hidden Cost of Untested Code


**The "I Know It Works" Illusion**
- Testing code mentally or with temporary `print()` statements:
    - Works **today** and for this **one case**,
    - But tomorrow may be different: a colleague changes a dependency, or the code is refactored.
        - (That small, manual `print()` statement check is lost or bypassed.)

**Take-Home Message**
- Untested code is problematic:
    - Reduced confidence in it
    - Results in fear of making changes
    - Costly debugging

<br>

- Unit testing <b>flips</b> this dynamic
    - Gives you <b>confidence</b> to <b>refactor</b>, upgrade, and scale your code
        - If you break something, the tests will catch it
    - <b>Debugging</b> is <b>faster</b> and <b>easier</b>

<hr style="border:2px solid gray"></hr>

# Unit Tests

- An <b>'automated'</b> procedure to verify a **small**, **isolated** code piece (i.e., a **unit**).
    - Make sure that the unit works as intended under different conditions.
 
## Unit Tests Characteristics

- <font color='DodgerBlue'>**Isolation**</font>
    - Focuses on the <b>smallest</b> testable <b>part</b> of the <b>code</b> (e.g., a single function).
    - It must be <b>run</b> in <b>isolation</b> (i.e., it doesn't rely on external factors like databases).

- <font color='DodgerBlue'>**Assertion / Raise Statements**</font>
    - Every unit test contains one or more <b>assertions/raise statements</b>.
        - An assertion is a conditional check (e.g., `assert calculated_value == expected_value`).
        - If it fails, the <b>test stops</b> and <b>reports an error</b>.

- <font color='DodgerBlue'>Automation</font>
    - The test itself is code.
    - It can be run instantly and repeatedly by a testing framework (e.g., <b>pytest</b>, unittest).
    - This is crucial for speed and consistency.

- <font color='DodgerBlue'>Speed</font>
    - Unit tests must execute quickly (milliseconds).
        - They will be run constantly by developers (or automated build systems).

### Assert vs. Raise

- Comparing <font color='DodgerBlue'>two floats</font> to be within a <font color='DodgerBlue'>tolerance</font> range.

In [1]:
tolerance = 0.001

calculated_number = 0.115
expected_number = 0.117

#### Standard `assert`: raises an AssertionError

Condition
- `abs(calculated_number - expected_number)` must be **less than** the `tolerance` value
- If `True`, then continue.
- If `False`, then **raise customized error**:

In [2]:
abs(calculated_number - expected_number) <= tolerance

False

In [3]:
assert abs(calculated_number - expected_number) <= tolerance, "Values are too far apart."

AssertionError: Values are too far apart.

#### Custom `raise`: Checks the condition, explicitly raises an exception on failure

- `abs(calculated_number - expected_number)` is **greater than** the `tolerance` value
- If `True`, then **raise error**:
- If `False`, then continue.

In [4]:
abs(calculated_number - expected_number) >= tolerance

True

In [5]:
if abs(calculated_number - expected_number) >= tolerance:
    raise AssertionError(f"Error: Expected value was {expected_number:.3f}, but obtained {calculated_number:.3f}.")

AssertionError: Error: Expected value was 0.117, but obtained 0.115.

## Example

In [6]:
def calculate_grade(score, max_score=100):
    """ Calculates the percentage grade from a score. """

    percentage = (score / max_score) * 100

    return percentage

**Manual check**

In [8]:
print(calculate_grade(85))

85.0


Look correct - great!

## Unit Test - Raising Errors

In [9]:
def test_calculate_grade():
    """ Verifies the function's output for known, good data. """

    # 1. Standard case check (80/100 should be 80.0)
    assert calculate_grade(80) == 80.0, "The calculate_grade function does not work for when max_score is 100."

    # 2. Edge case check (max_score is not 100)
    assert calculate_grade(25, max_score=50) == 50.0, "The calculate_grade function does not work for when max_score is not 100."

    print("Test Passed.")

In [10]:
test_calculate_grade()

Test Passed.


We will add a `try`-`except` statement (for comparison to a later case where the test fails):

In [11]:
try:
    test_calculate_grade()
except AssertionError as error:
    print("TEST FAILED! The test caught a bug that the simple manual check missed.")
    print(f"Assert Failure Details: {error}")
    print("This is the power of a Unit Test: immediate feedback and regression prevention.")

Test Passed.


### Scenario
Now, let's say in 10 months a colleague moves the 100 to a global position (i.e. outside the user-defined function).

In [12]:
def calculate_grade(score, max_score=100):
    """ Buggy - calculates the percentage grade from a score. """

    # BUG: Forgot to multiply by 100
    percentage = (score / max_score) 

    return percentage

We will still use our `test_calculate_grade()` function, but now it will test the <b>new buggy</b> `calculate_grade()` function.

In [13]:
test_calculate_grade()

AssertionError: The calculate_grade function does not work for when max_score is 100.

Now we will introduce the `try-except` statement for code <b>error handling</b>:

In [14]:
try:
    test_calculate_grade()
except AssertionError as error:
    print("TEST FAILED! The test caught a bug that the simple manual check missed.")
    print(f"Assert Failure Details: {error}")
    print("This is the power of a Unit Test: immediate feedback and regression prevention.")

TEST FAILED! The test caught a bug that the simple manual check missed.
Assert Failure Details: The calculate_grade function does not work for when max_score is 100.
This is the power of a Unit Test: immediate feedback and regression prevention.


<hr style="border:2px solid gray"></hr>

## The `try`-`except` Statement - to Handle Errors
- Tells your code to try something, and
- then tells it what to do if it fails based on an <b>exception type</b> (i.e., an error).

 
#### Strengths:
1. The code will <font color='DodgerBlue'>**continue**</font> to run, even when it encounters a problem.

   - This prevents the program from crashing (e.g., if a test fails).<br><br>

2. <font color='DodgerBlue'>**Faster** than if statements</font> for when <font color='DodgerBlue'>majority of the planned tasks are **expected** to be **successful**</font> (i.e., they don't encounter an exception)

3. Built-in exceptions: https://docs.python.org/3/library/exceptions.html#bltin-exceptions

<br><br>
**Simple Example**

In [15]:
try:
    print(5/0)
except ZeroDivisionError:
    print(f"Error: You can't have a zero in the denominator.")

Error: You can't have a zero in the denominator.


**A More Sophisticated Example**

Let's set up a <font color='DodgerBlue'>division calculator</font> that allows users to input numbers and quit at any time using while and if loops (to demonstrate via a comparison of code).

1. First, set something up without `try`-`except` in order to see its advantage later.
2. Second, do the same thing with `try`-`except`

<font color='DodgerBlue'>Demonstrate the following</font>:
1. normal operation
2. exiting by typing 'q'
3. <font color='Red'>O</font> (i.e., a capital alphabet letter "O", as in "O"ktoberfest (also demonstrates traceback error))

Without `try`-`except` statement:

In [17]:
## print('Type two numbers that you want to be divided.')
print("Type 'q' to quit.")
print()

while True:
    numerator = input('Numerator = ')
    if numerator == 'q':
        break

    denominator = input('Denominator = ')
    if denominator == 'q':
        break

    if denominator == '0':
        print("You can't have a zero in the denominator.")
        break

    answer = float(numerator)/float(denominator)
    print(f'Answer for {numerator}/{denominator} = {answer}\n')

Type 'q' to quit.



Numerator =  O
Denominator =  4


ValueError: could not convert string to float: 'O'

Modify the above code to use a **`try`-`except` statement**, and try it with <font color='Red'>O</font>.

**Note**: <font color='DodgerBlue'>Multiple `except` conditions via a **tuple**</font>:<br>
`except (ZeroDivisionError, ValueError):`
- `ZeroDivisionError` when the denominator is zero
- `ValueError` for when a string is given as an input

In [18]:
print('Type two numbers that you want to be divided.')
print("Type 'q' to quit.")
print()

while True:
    numerator = input('Numerator = ')
    if numerator == 'q':
        break

    denominator = input('Denominator = ')
    if denominator == 'q':
        break

    try:
        answer = float(numerator)/float(denominator)
        print(f'Answer for {numerator}/{denominator} = {answer}\n')

    except (ZeroDivisionError, ValueError):
        print('Your input was either not a number, or you are dividing by a zero.\n')

Type two numbers that you want to be divided.
Type 'q' to quit.



Numerator =  8
Denominator =  4


Answer for 8/4 = 2.0



Numerator =  6
Denominator =  O


Your input was either not a number, or you are dividing by a zero.



Numerator =  q


<font color='DodgerBlue'>Now the code continues to run, even though an error was raised!</font>

### Warning - Be Careful when Using `try-except` Statements
- They will mask or ignore potential bugs

<b>Scenario</b>: Creating a dungeon crawler game

#### Bad Practice: Demonstrates Error Hiding and False Success

In [19]:
players = {1: None, 2: None, 3: None, 4: None, 5: None}
equipment_list = ['ax', 'potion', 'wand', 'dust', 'sword']

index = 0

while index <= 5:
    index += 1
    try:
        players[index + 1] = equipment_list[index] 
    except IndexError:
        break  # Stop the loop, hiding the fact that not all players were assigned.

# The data is corrupt/incomplete, but the program claims success.
print("Program finished execution.")
print("Success: Program finished execution with all players assigned.")

Program finished execution.
Success: Program finished execution with all players assigned.


In [20]:
print(f"Player data (corrupted):\n{players}")

Player data (corrupted):
{1: None, 2: 'potion', 3: 'wand', 4: 'dust', 5: 'sword'}


<b>Notice</b>: <b>Player 1</b> has <b>no</b> equipment assigned.

<br>

#### Better Practice: Allows logical errors to stop the algorithm.

- Allow the program to error, thus notifying the programmer:

In [21]:
players = {1: None, 2: None, 3: None, 4: None, 5: None}
equipment_list = ['ax', 'potion', 'wand', 'dust', 'sword']

index = 0

while index <= 5:
    # Logical bug: index will reach 5, which is beyond the equipment_list range.
    # The assignment below will fail when index = 5.
    index += 1
    players[index + 1] = equipment_list[index] 
    
print("Program finished execution.")
print("Success: Program finished execution with all players assigned.")

IndexError: list index out of range

<br>

#### Best Solution: correct, clear, and robust

To show how the problem could be coded well:

In [22]:
players = {1: None, 2: None, 3: None, 4: None, 5: None}
equipment_list = ['ax', 'potion', 'wand', 'dust', 'sword']

for index in range(len(equipment_list)): # Correctly iterates from 0 to 4
    players[index + 1] = equipment_list[index] 

# Verification
if None in players.values():
    print("Problem: Program finished execution with UNASSIGNED players.")
else:
    print("Program finished execution.")
    print("Success: Program finished execution with all players assigned.")

Program finished execution.
Success: Program finished execution with all players assigned.


In [23]:
print(f"Player data (uncorrupted):\n{players}")

Player data (uncorrupted):
{1: 'ax', 2: 'potion', 3: 'wand', 4: 'dust', 5: 'sword'}


<hr style="border:2px solid gray"></hr>

<h1 align='center'>Test Driven Development (<font color='DodgerBlue'>TDD</font>)
    
<h2 align='center'> using Unit Tests

https://docs.python.org/3/library/unittest.html

## Test Driven Development - writing tests before you write your production code
1. Ensures proper and directed functionality of your code
 - creating **concise** code that does a **single** thing (e.g., <font color='DodgerBlue'>**user-defined functions**</font>)
2. Helps you plan your code - what do you **actually want** to do (critical thinking)
3. Reduces **errors**
4. Ensures **reproducibility**
5. Helps to ensure a code's **long life**

## The Traditional Workflow Concept (write test -> write production code)
1. Write a failing test
2. Run and ensure failure
3. Write code to pass
4. Run and ensure passing
5. Refactor (i.e., restructure/clean up code without changing it final result)
6. Redo steps 1-5

However, the **sequence** of test and production code development **might not matter** [1]:
1. Write **test** -> write **production** code, or
2. Write **production** code -> write **test** -> **refactor** production code

<font color='DodgerBlue'>**What is important is that unit tests are included!**</font>

## Scientific and Data Research
It is **CRITICAL** that:
1. The correct results are generated
2. The results are reproducible

#### Assert statements that can be used in unittest library

https://docs.python.org/3/library/unittest.html#module-unittest


| Method | Checks that | New in |
|:------|:-:|:-:|
| assertEqual(a, b) | a == b | |
| assertNotEqual(a, b)| a != b | |
| assertTrue(x) | bool(x) is True | |
| assertFalse(x) | bool(x) is False | |
| assertIs(a, b) | a is b | 3.1 |
| assertIsNot(a, b) | a is not b | 3.1 |
| assertIsNone(x) | x is None | 3.1 |
| assertIsNotNone(x) | x is not None | 3.1 |
| assertIn(a, b) | a in b | 3.1 |
| assertNotIn(a, b) | a not in b | 3.1 |
| assertIsInstance(a, b) | isinstance(a, b) | 3.2 |
| assertNotIsInstance(a, b) | not isinstance(a, b) | 3.2 |
| | | |
| | | |
| assertAlmostEqual(a, b) | round(a-b, 7) == 0 | |
| assertNotAlmostEqual(a, b) | round(a-b, 7) != 0 | |
| assertGreater(a, b) | a > b | 3.1 |
| assertGreaterEqual(a, b) | a >= b | 3.1 |
| assertLess(a, b) | a < b | 3.1 |
| assertLessEqual(a, b) | a <= b | 3.1 |
| assertRegex(s, r) | r.search(s) | 3.1 |
| assertNotRegex(s, r) | not r.search(s) | 3.2 |
| assertCountEqual(a, b) | a and b have the same elements in the same number, regardless of their order. | 3.2 |


| Method | Used to compare | New in|
|:------|:-:|:-:|
| assertMultiLineEqual(a, b) | strings | 3.1 |
| assertSequenceEqual(a, b) | sequences | 3.1 |
| assertListEqual(a, b) | lists | 3.1 |
| assertTupleEqual(a, b) | tuples | 3.1 |
| assertSetEqual(a, b) | sets or frozensets | 3.1 |
| assertDictEqual(a, b) | dicts | 3.1 |

Demonstrate the following two scenarios:
1. Scenario 1: the unit test runs with everything correct
2. Scenario 2: the unit test runs, but with errors
     - A new `assertEqual` is added

**Note**: We will include <font color='DodgerBlue'>additional assert statements just to demonstrate how the output of a unit test looks like</font>, even though it is not relevant to our user-defined function.

**Scenario 1**: the unit test runs with everything <font color='DodgerBlue'>correctly</font>

Define a user-defined function to demo how that is done:

In [24]:
def hello_world():
    return 'hello world'

**Note**: `isupper()`

- `str.isupper()`: Return `True` if all characters in a given string are uppercase, otherwise it is `False`.
    
    - https://docs.python.org/3/library/stdtypes.html

In [25]:
import unittest


class MyFirstUniTTests(unittest.TestCase):

    def test_isEqual(self):
        self.assertEqual(hello_world(), 'hello world')

    def test_isLess(self):
        self.assertLess(5, 10)

    def test_isLessEqual(self):
        self.assertLessEqual(10, 10)

    def test_isUpperTrue(self):
        self.assertTrue('FOO'.isupper())

    def test_isUpperFalse(self):
        self.assertFalse('Foo'.isupper())


## Normal usage (in a .py script)
#if __name__ == '__main__':
#    unittest.main()

## For usage in jupyter and colaboratory (due to the kernel)
if __name__ == '__main__':
    unittest.main(argv=['ignored', '-v'], exit=False)

test_isEqual (__main__.MyFirstUniTTests.test_isEqual) ... ok
test_isLess (__main__.MyFirstUniTTests.test_isLess) ... ok
test_isLessEqual (__main__.MyFirstUniTTests.test_isLessEqual) ... ok
test_isUpperFalse (__main__.MyFirstUniTTests.test_isUpperFalse) ... ok
test_isUpperTrue (__main__.MyFirstUniTTests.test_isUpperTrue) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.006s

OK


**Scenario 2**: the unit test runs, but some <font color='red'>errors</font> occur (i.e., in the **first test**):

In [26]:
class MyFirstUniTTests(unittest.TestCase):

    def test_fail(self):
       self.assertEqual(hello_world(), 'bye world')

    def test_isEqual(self):
        self.assertEqual(hello_world(), 'hello world')

    def test_isLess(self):
        self.assertLess(5, 10)

    def test_isLessEqual(self):
        self.assertLessEqual(10, 10)

    def test_isUpperTrue(self):
        self.assertTrue('FOO'.isupper())

    def test_isUpperFalse(self):
        self.assertFalse('Foo'.isupper())


if __name__ == '__main__':
    unittest.main(argv=['ignored', '-v'], exit=False)

test_fail (__main__.MyFirstUniTTests.test_fail) ... FAIL
test_isEqual (__main__.MyFirstUniTTests.test_isEqual) ... ok
test_isLess (__main__.MyFirstUniTTests.test_isLess) ... ok
test_isLessEqual (__main__.MyFirstUniTTests.test_isLessEqual) ... ok
test_isUpperFalse (__main__.MyFirstUniTTests.test_isUpperFalse) ... ok
test_isUpperTrue (__main__.MyFirstUniTTests.test_isUpperTrue) ... ok

FAIL: test_fail (__main__.MyFirstUniTTests.test_fail)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_64572/3283447063.py", line 4, in test_fail
    self.assertEqual(hello_world(), 'bye world')
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: 'hello world' != 'bye world'
- hello world
+ bye world


----------------------------------------------------------------------
Ran 6 tests in 0.012s

FAILED (failures=1)


## PyTest (state-of-the-art)

1. A command-line driven testing approach (e.g., via bash shell)
2. Simplifies and helps organize unit tests
    - done by creating **user-defined functions** for **each test** that you want to do

https://docs.pytest.org/en/stable/

In [27]:
%%writefile test_sum.py
''' The following will be created:
        1. Four unit test functions, where
            a. First 3 will pass, and
            b. Last 1 will fail.
'''

def test_pass_add_list_1():
    ''' 1st unit test.
    '''
    test_list = [1, 2, 3, 4]
    assert sum(test_list) == 10


def test_pass_add_list_2():
    ''' 2nd unit test.
    '''
    test_list = [1, 2, 3, 4, 5]
    assert sum(test_list) == 15


def test_pass_add_list_3():
    ''' 3rd unit test with print statements.
    '''
    print('PRINT STATEMENT FOR FAILING TEST FUNCTION.')
    print('Another statement')

    test_list = [1, 2, 3, 4, 5, 6]
    assert sum(test_list) == 21


def test_fail_add_list_4():
    ''' 4th unit test
        Should Fail
    '''    
    test_list = [1, 2, 3, 4, 5, 6]
    assert sum(test_list) == 0

Writing test_sum.py


PyTest will give the following output:
- `.` (dot) = test <font color='DodgerBlue'>passed</font>
- `F` =  test has <font color='DodgerBlue'>failed</font>
- `E` =  test raised an <font color='DodgerBlue'>unexpected exception</font>

In [28]:
! pytest test_sum.py

=========================================================== test session starts ============================================================
platform linux -- Python 3.13.2, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/karl/Scientific_Programming_Course
plugins: anyio-4.8.0
collected 4 items                                                                                                                          

test_sum.py ...F                                                                                                                     [100%]

================================================================= FAILURES =================================================================
___________________________________________________________ test_fail_add_list_4 ___________________________________________________________

    def test_fail_add_list_4():
        ''' 4th unit test
            Should Fail
        '''
        test_list = [1, 2, 3, 4, 5, 6]
>       assert sum(test_list) 

**Output**
1. The first three test passed
2. The fourth test fails
3. A traceback is given concerning the error
4. None of the print statements are seen

<font color='DodgerBlue'>To see print commands</font> within the user-defined functions, <font color='DodgerBlue'>use `-s` option</font>:

(`-s` is a shortcut for `--capture=no` - see `pytest --help`)

In [29]:
! pytest test_sum.py -s

=========================================================== test session starts ============================================================
platform linux -- Python 3.13.2, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/karl/Scientific_Programming_Course
plugins: anyio-4.8.0
collected 4 items                                                                                                                          

test_sum.py ..PRINT STATEMENT FOR FAILING TEST FUNCTION.
Another statement
.F

================================================================= FAILURES =================================================================
___________________________________________________________ test_fail_add_list_4 ___________________________________________________________

    def test_fail_add_list_4():
        ''' 4th unit test
            Should Fail
        '''
        test_list = [1, 2, 3, 4, 5, 6]
>       assert sum(test_list) == 0
E       assert 21 == 0
E        +  where 21 = sum([1, 2, 

Clean up directory:

In [ ]:
! rm test_sum.py

## Example of Notebook Function and PyTesting

Create a User-defined Function

In [30]:
def calculate_area(length, width):
    """ Calculate the area of a rectangle.
    """
    if length <= 0 or width <= 0:
        raise ValueError("Dimensions must be positive.")
    return length * width

Create PyTest

In [31]:
%%writefile test_calculate_area.py
import pytest

def calculate_area(length, width):
    """ Calculate the area of a rectangle.
    """
    if length <= 0 or width <= 0:
        raise ValueError("Dimensions must be positive.")
    else:
        return length * width


def test_positive_dimensions():
    # Test case with standard positive inputs
    assert calculate_area(5, 4) == 20


def test_square_dimensions():
    # Test case for a square (equal sides)
    assert calculate_area(10, 10) == 100


def test_zero_or_negative_dimensions():
    # Test case for invalid input using pytest.raises
    with pytest.raises(ValueError):
        calculate_area(-1, 5)
    with pytest.raises(ValueError):
        calculate_area(0, 5)

Writing test_calculate_area.py


In [32]:
! pytest test_calculate_area.py

=========================================================== test session starts ============================================================
platform linux -- Python 3.13.2, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/karl/Scientific_Programming_Course
plugins: anyio-4.8.0
collected 3 items                                                                                                                          

test_calculate_area.py ...                                                                                                           [100%]

============================================================ 3 passed in 0.03s =============================================================


In [33]:
! rm test_calculate_area.py

<hr style="border:2px solid gray"></hr>

## Unit Tests in the Age of Generative AI (GenAI)

Topics to Consider:
1. **Testing GenAI-generated code** to ensure correctness
2. Using **GenAI** to help **generate unit tests**

In [ ]:
from IPython import display

display.Image("https://imgs.xkcd.com/comics/machine_learning.png")

## GenAI and Unit Tests [2, 3]

"Generative AI (GenAI) may reduce the extra effort imposed by TDD." [2]

"**GenAI** can be efficiently used in TDD, but it **requires supervision** on the **quality** of the **code produced**. In some cases, it can **mislead** non-expert **developers** and **propose solutions that change tests rather than the production code**, which may remain buggy, to **make tests pass**."

**GenAI** to **create Unit Tests**: <font color='DodgerBlue'>**Cutting Edge Idea**</font>, but
- <font color='DodgerBlue'>There is very little experience</font> on how to do this **properly**
- Example, the **name** of a **user-defined** function can iinfluence the <font color='DodgerBlue'>GenAI to create **wrong code**</font> [3]

### Conceptual Approaches [4]
1. <font color='magenta'>**GenAI**<sub>IDEA</sub></font> \& <font color='magenta'>**GenAI**<sub>IMPL</sub></font>
    - GenAI generates the testing **ideas** (i.e., test cases themselves)
    - GenAI generates the **test code** (i.e., implementation of test cases)

2. <font color='magenta'>**GenAI**<sub>IDEA</sub></font> \& <font color='DodgerBlue'>**Human**<sub>IMPL</sub></font>
    - GenAI generates the testing **ideas**
    - Human writes the actual test code

3. <font color='DodgerBlue'>**Human**<sub>IDEA</sub></font> \& <font color='magenta'>**GenAI**<sub>IMPL</sub></font> (**Recommended**)
    - Human generates the testing **ideas**
    - GenAI generates the **test code**


### Potential Benefits from using GenAI [4]
1. **Time-saving**
2. Reduced cognitive load (a double-edged sword)

### Potential Drawbacks from using GenAI [4]
1. **Lack of Ownership**
   - feeling less accountable for code's correctness
   - more inclined to accept the output without critical evaluation

2. **Lack of Quality**
   - Misses edge cases or complex logic
   - Partially aligned test with the user-defined functional requirements
   - Unpolished/duplicated code/tests

3. **Lack of Trust**
   - Inconsistencies in GenAI outputs across similar prompts
   - Trivial test cases
   - Redundant or irrelevant tests that added noise rather than insight


## Testing GenAI-Generated Code

<b>LLMs</b> are capable of generating <b>entire functions</b>, which means that
- Code is written <b>faster</b>, but
- <b>Not necessarily safer</b> or more <b>correct</b>.
    
<b>Unit tests</b> serve as the essential
- <b>Safety net</b>
- <b>Verification step</b>

### Final Thoughts about Unit Tests
1. Source of <b>Truth</b>

    - Verifying AI-Generated Code (via <b>LLMs</b>):
        - excellent at writing <b>syntactically correct</b> code,
        - can <b>miss edge cases</b>, or
        - <b>misinterpret</b> complex <b>domain logic</b>.
    - Unit tests
        - Deterministic <font color='DodgerBlue'><b>source of truth</b></font> that **verifies** an AI's output.
        - Guards <b>against "hallucinations"</b>.

2. Maintaining <b>Stability</b> During Rapid Code <b>Iterations</b>

    - Safety Net for Refactoring
        - GenAI tools <b>enable</b> rapid <b>refactoring / restructuring</b>
        - <b>Unit test suite</b> - allows <b>developers</b> to <font color='DodgerBlue'><b>confirm</b> AI-driven refactoring</font> (preventing bugs).

    - Faster Debugging (Pinpointing the Error) of complex AI-generated code

3. The Core of Effective Prompt Engineering for AI

    - Test-Driven Prompting
        - The <b>unit test</b> file itself <font color='DodgerBlue'><b>becomes a powerful prompt</b></font> you can give an <b>LLM</b>.
            - "Write Python code that makes this unit test pass" (vs. less precise/effective "Write a function to do X.")

    - Defining Edge Cases
        - Explicitly define expected behavior for all inputs (e.g., negative numbers, empty arrays, null values).
        - Might results in more <font color='DodgerBlue'><b>robust AI code</b></font>.

4. Knowledge Transfer
    - Onboarding and Documentation - <font color='DodgerBlue'>Unit tests are a form of living documentation</font>
        - <b>New</b> developers running unit tests and <b>reading their inputs/assertions</b> is an efficient way to understand the intended <b>code's functionality</b> and <b>requirements</b>

---
# Take Home Messages

In <b>scientific research</b>, it's <b>very important</b> that code produces <b>correct</b> and <b>reproducible</b> results.

1. Importance of Testing and Untested Code
    - Untested code is problematic because it:
        - <b>Reduced confidence</b>
        - Causes <b>fear</b> of making <b>changes</b> (refactoring), and
        - Is <b>costly</b> debugging
    
    - Testing <b>flips</b> this dynamic.

    - "I Know It Works" illusion is dangerous
        - <b>Manual checks work temporarily</b> (e.g., print statement), but can change tomorrow (e.g., refactoring)
        - Can easily break functionality without detection

- <b>Unit Tests - Raising Errors</b>

    - A unit test: an 'automated' procedure to verify a small, <b>isolated code piece</b> (e.g., a function) works as intended under different conditions.

    - Unit Test's Key Characteristics:
        - <b>Isolation</b>: Focuses on the <b>smallest testable part of the code</b>, and must run without external dependencies (e.g., databases).
        - <b>Assertion / Raise Statements</b>: Every test contains these checks that <b>stop</b> and <b>report</b> an <b>error</b> if they fail.
        - <b>Automation</b>: The test runs without interventions, and can do so through a framework (e.g., pytest, unittest).
        - <b>Speed</b>: Executes very quickly since they are constantly rerun.

- <b>`try-except` - Error Handling</b>

    - `try-except` statement tells your code to
        - try a block of code, and
        - then specifies what to do if an exception (error) of a certain type occurs.

    - Strengths:
        - The <b>code continues to run</b> even after <b>encountering a problem</b>.
        - Faster than using if statements when the majority of tasks are expected to be successful.

    - Weakness:
         - They can <b>mask or ignore potential bugs</b>
             - Leading to corrupt or incomplete data while the program suggests otherwise (i.e., Error Hiding).
         - Better practice: <b>allow logical errors to stop the algorithm</b>, thus notifying the programmer.

- <b>Test-Driven Development</b> (TDD)

    - TDD involves writing tests before you write your production code.

    - Benefits:

        - Ensures proper and directed functionality - <b>leads to concise code</b> that does a single thing.

        - Helps <b>plan your code</b> and <b>encourages critical thinking</b> about what you actually want to do.

        - <b>Reduces errors, ensures reproducibility</b>, and helps ensure a code's long life (<b>maintainability, sustainability</b>).

    - TDD Workflow Concept
 
        - Write a failing test,
        - Run and ensure failure,
        - Write code to pass,
        - Run and ensure passing,
        - Refactor (restructure/clean up code without changing its final result), and
        - Redo

- <b>PyTest</b> Tool

    - PyTest is a command-line driven testing framework that <b>simplifies and organizes unit tests</b>
        - via <b>creating separate user-defined functions for each test</b>.

    - The output symbols help quickly assess test results
 
        - `.` for passed,
        - `F` for failed, and
        - `E` for an unexpected exception

- <b>Unit tests</b> are becoming <b>more important</b> in the age of <b>GenAI</b>

    - <font color='DodgerBlue'>Recommened Interaction: **Human**<sub>IDEA</sub></font> \& <font color='magenta'>**GenAI**<sub>IMPL</sub></font>
    - GenAI usage is still an uncharted territory
        - no **"absolutes"** yet
        - output is **prone to error**
    - Critical thinking is still needed